### Data Merge

##### Declarations

In [1]:
#Importing utilities file with library imports and helper functions
%run "utils.ipynb"

In [2]:
#Loading input pickle files
df_income_1901_edited=pd.read_pickle(input_path+"\\df_income_1901_edited.pkl")
df_health_outcomes_edited=pd.read_pickle(input_path+"\\df_health_outcomes_edited.pkl")
df_sdoh_edited=pd.read_pickle(input_path+"\\df_sdoh_edited.pkl")
df_state_politics_edited=pd.read_pickle(input_path+"\\df_state_politics_edited.pkl")
df_state_public_spend_edited=pd.read_pickle(input_path+"\\df_state_public_spend_edited.pkl")
df_acs_edited=pd.read_pickle(input_path+"\\df_acs_edited.pkl")
df_zip2st_edited=pd.read_pickle(input_path+"\\df_zip2st_edited.pkl")
df_urban_rural_edited=pd.read_pickle(input_path+"\\df_urban_rural_edited.pkl")

In [3]:
#Printing shapes to check which file needs most attention for data compression
for df in [df_income_1901_edited, df_health_outcomes_edited, df_sdoh_edited, df_state_politics_edited, 
           df_state_public_spend_edited, df_acs_edited, df_urban_rural_edited]:
    print(df.shape)

(33774, 15)
(1169992, 10)
(291024, 4)
(57, 3)
(255, 3)
(33120, 14)
(33178, 4)


##### ZIP level data

In [4]:
[df_income_1901_edited.ZIP.nunique(), df_health_outcomes_edited.ZIP.nunique(), df_sdoh_edited.ZIP.nunique(), 
 df_acs_edited.ZIP.nunique(), df_urban_rural_edited.ZIP.nunique()]

[33774, 32409, 32336, 33120, 33178]

In [5]:
df_income_1901_essential=df_income_1901_edited[['ZIP', 'Estimate Households Total', 'Percent Households lt 10k', 
                                                'Estimate Households Median income (dollars)', 'Estimate Families Total',
                                                'Estimate Families Median income (dollars)', 'Estimate Nonfamily households Total',
                                                'Estimate Nonfamily households Median income (dollars)', 'Percent Households that are Families']]
df_income_1901_essential.shape

(33774, 9)

In [6]:
# Selecting essential columns from df_health_outcomes_edited and reshaping it to wide format with pivot
df_health_outcomes_essential = df_health_outcomes_edited[
    ["ZIP", "TotalPopulation", "Short_Question_Text", "Data_Value"]
]

df_health_outcomes_essential_wide = (
    df_health_outcomes_essential.pivot(
        index=["ZIP", "TotalPopulation"],
        columns="Short_Question_Text",
        values="Data_Value",
    )
    .reset_index()
    .rename_axis(None, axis=1)
)


# Uncomment to display the first few rows of the reshaped DataFrame
# display(df_health_outcomes_essential_wide.head())
# Displaying the shape of the reshaped DataFrame

df_health_outcomes_essential_wide.shape

(32409, 39)

In [7]:
# Reshaping df_sdoh_edited to wide format with pivot, using 'ZIP' and 'TotalPopulation' as index, 'Measure' as columns, and 'Data_Value' as values
df_sdoh_essential_wide = (
    df_sdoh_edited.pivot(
        index=["ZIP", "TotalPopulation"], columns="Measure", values="Data_Value"
    )
    .reset_index()
    .rename_axis(None, axis=1)
)


# Uncomment to display the first few rows of the reshaped DataFrame
# display(df_sdoh_essential_wide.head())

# Displaying the shape of the reshaped DataFrame
df_sdoh_essential_wide.shape

(32336, 11)

##### State level data

In [8]:
# Checking if any of the state level data needs to be pivoted
print(
    "df_state_politics_edited",
    df_state_politics_edited.shape,
    df_state_politics_edited.State.nunique(),
)
print("df_zip2st_edited", df_zip2st_edited.shape, df_zip2st_edited.ZIP.nunique())
print(
    "df_state_public_spend_edited",
    df_state_public_spend_edited.shape,
    df_state_public_spend_edited["State Name"].nunique(),
)

df_state_politics_edited (57, 3) 57
df_zip2st_edited (39368, 2) 39368
df_state_public_spend_edited (255, 3) 51


In [9]:
# Turns out state_public_spend_edited will need to be pivoted as it has multiple values of year for each state
df_state_public_spend_essential_wide = (
    df_state_public_spend_edited.pivot(
        index=["State Name"], columns="TimeFrame", values="Data"
    )
    .reset_index()
    .rename_axis(None, axis=1)
    .add_prefix("spend_")
    .rename(columns={"spend_State Name": "State Name"})
)
df_state_public_spend_essential_wide.head()

,State Name,spend_2017,spend_2018,spend_2019,spend_2020,spend_2021
0,Alabama,56.43214,57.23,54.25,46.81,52.35
1,Alaska,114.43666,96.52,98.25,215.15,90.17
2,Arizona,8.83289,9.66,15.25,24.34,15.32
3,Arkansas,52.29329,50.57,48.23,45.90,42.94
4,California,61.69589,65.71,71.89,69.87,78.47


In [10]:
#Geographic information (Lat-Long)
geocoords=df_health_outcomes_edited[['ZIP', 'longitude', 'latitude']].drop_duplicates()
geocoords.shape, geocoords.ZIP.nunique()

((32409, 3), 32409)

##### Creating combined dataset

In [11]:
# Combined_dataset_zip contains ZIP level features.
# Combined_dataset_state contains State level features.
# Combined_dataset contains all features

# Merging dataframes for ZIP level features
combined_dataset_zip = pd.merge(
    pd.merge(
        pd.merge(
            pd.merge(
                df_income_1901_essential,
                df_health_outcomes_essential_wide,
                on="ZIP",
                how="outer",
            ),
            df_sdoh_essential_wide,
            on="ZIP",
            how="outer",
        ),
        df_acs_edited,
        on="ZIP",
        how="outer",
    ),
    df_urban_rural_edited,
    on="ZIP",
    how="left",
)

# Merging dataframes for State level features
combined_dataset_state = pd.merge(
    pd.merge(
        df_state_politics_edited,
        df_state_public_spend_essential_wide,
        on="State Name",
        how="outer",
    ),
    df_zip2st_edited,
    on="State",
    how="outer",
)

# Merging ZIP and State level features
combined_dataset = pd.merge(
    pd.merge(combined_dataset_zip, combined_dataset_state, on="ZIP", how="outer"),
    geocoords,
    on="ZIP",
    how="left",
)

# Print the shape of the combined dataset
combined_dataset.shape

(39503, 83)

In [12]:
combined_dataset.sample(5)

,ZIP,Estimate Households Total,Percent Households lt 10k,Estimate Households Median income (dollars),Estimate Families Total,Estimate Families Median income (dollars),Estimate Nonfamily households Total,Estimate Nonfamily households Median income (dollars),Percent Households that are Families,TotalPopulation_x,All Teeth Lost,Annual Checkup,Any Disability,Arthritis,Binge Drinking,COPD,Cancer (except skin),Cervical Cancer Screening,Cholesterol Screening,Chronic Kidney Disease,Cognitive Disability,Colorectal Cancer Screening,Core preventive services for older men,Core preventive services for older women,Coronary Heart Disease,Current Asthma,Current Smoking,Dental Visit,Depression,Diabetes,General Health,Health Insurance,Hearing Disability,High Blood Pressure,High Cholesterol,Independent Living Disability,Mammography,Mental Health,Mobility Disability,Obesity,Physical Health,Physical Inactivity,Self-care Disability,Sleep <7 hours,Stroke,Taking BP Medication,Vision Disability,TotalPopulation_y,Crowding among housing units,Housing cost burden among households,No broadband internet subscription among households,No high school diploma among adults aged 25 years or older,Persons aged 65 years or older,Persons living below 150% of the poverty level,Persons of racial or ethnic minority status,Single-parent households,Unemployment among people 16 years and older in the labor force,Estimate!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),Percent!!RACE!!Total population,Percent!!RACE!!Total population!!One race,Percent!!RACE!!Total population!!One race!!Black or African American,Percent Margin of Error!!RACE!!Total population!!One race!!Black or African American,Percent!!RACE!!Total population!!One race!!American Indian and Alaska Native,Percent Margin of Error!!RACE!!Total population!!One race!!American Indian and Alaska Native,Percent!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander,Percent Margin of Error!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race),Percent Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race),Estimate!!Total housing units,Urban+Rural,Urban,Percent_Urban,State Name,State,Political Affiliation (2008-2020 presidential elections),spend_2017,spend_2018,spend_2019,spend_2020,spend_2021,longitude,latitude
37474,76122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Texas,TX,Deep Red,21.57633,16.92,17.16,20.41,17.91,NaN,NaN
13874,42240,16360.0,8.2,46116.0,10030.0,59838.0,6330.0,27794.0,0.613081,42479.0,22.4,76.6,43.4,32.3,13.3,11.0,6.8,79.9,83.1,3.6,20.6,70.1,40.9,34.4,7.6,12.6,24.2,49.1,28.6,14.6,26.4,9.1,9.2,42.5,36.4,12.6,74.7,19.8,22.6,39.5,16.2,33.4,5.9,40.9,4.1,80.7,8.0,41318.0,0.8,31.5,24.8,14.8,17.0,33.2,32.6,10.2,7.3,41009.0,96.6,41009.0,95.7,23.7,1.8,0.4,0.3,0.3,0.2,2.9,0.9,19525.0,17322.0,13580.0,0.783974,Kentucky,KY,Deep Red,41.81652,34.23,35.46,35.89,29.64,-87.462338,36.878853
24723,70570,14651.0,10.5,43750.0,9400.0,53018.0,5251.0,21323.0,0.641594,38987.0,25.6,81.0,42.9,29.6,14.4,10.9,6.5,80.0,83.8,4.1,22.1,66.9,35.4,26.0,8.0,12.1,27.1,49.5,25.1,16.9,28.4,13.7,7.6,45.3,37.9,14.6,79.4,21.0,23.6,41.8,16.9,37.5,8.1,37.0,5.2,79.5,10.5,38236.0,3.8,24.6,32.4,19.3,16.6,38.2,61.5,9.5,5.9,39184.0,87.0,39184.0,99.1,58.5,2.3,0.3,0.2,0.1,0.2,1.9,0.8,17046.0,17500.0,12386.0,0.707771,Louisiana,LA,Deep Red,21.05356,23.98,24.81,25.58,41.21,-92.105017,30.532709
4254,14621,12987.0,11.5,32713.0,7248.0,37275.0,5739.0,23911.0,0.558097,33802.0,26.1,76.1,44.7,29.0,12.0,10.1,5.7,79.7,79.8,4.1,22.8,66.0,37.6,31.4,7.5,15.0,27.3,47.8,28.0,18.6,30.2,16.9,7.2,39.

In [13]:
combined_dataset['longitude'].isnull().sum()

7094

In [14]:
#Saving combined dataset
combined_dataset.to_pickle(output_path+"\\combined_dataset_anuvrat.pkl")
combined_dataset.to_csv(output_path+"\\combined_dataset_anuvrat.csv")

##### Checking if combined_dataset has values for all ZIPs

In [15]:
# % Missing values for each column. Function defined in helper functions notebook
perc_missing_num(combined_dataset)

Number of records in dataset: 39503
Missing records summary:


,column,number_missing,percent_missing
0,ZIP,3,0.007594
74,State,132,0.334152
73,State Name,132,0.334152
79,spend_2020,319,0.807534
78,spend_2019,319,0.807534
77,spend_2018,319,0.807534
76,spend_2017,319,0.807534
75,Political Affiliation (2008-2020 presidential ...,319,0.807534
80,spend_2021,2399,6.072956
1,Estimate Households Total,5729,14.502696


##### Dataset with strict removal of any missing value except % Urban

In [16]:
# Creating the dataset
combined_dataset_nona_strict = combined_dataset.dropna(
    subset=combined_dataset.columns.difference(
        ["Percent_Urban", "Urban+Rural", "Urban"]
    )
)

In [17]:
# % Missing values for each column. Function defined in helper functions notebook
#perc_missing_num(combined_dataset_nona_strict)

In [18]:
combined_dataset_nona_strict.shape

(24083, 83)

##### Checking which states are completely absent in the new dataset

In [19]:
# List of states in original dataset and dataset without any missing information
states_in_original=combined_dataset['State Name'].unique()
states_in_nona_strict=combined_dataset_nona_strict['State Name'].unique()

In [20]:
print("States not present in nona but present in original dataset")
for state in states_in_original:
    if state not in states_in_nona_strict: print(state)

States not present in nona but present in original dataset
Puerto Rico
nan
Rhode Island
Delaware
West Virginia
Florida
Kansas
Utah
American Samoa
Northern Mariana Islands
Trust Territories
Guam
Virgin Islands


In [21]:
#combined_dataset[combined_dataset['State Name'].isin(['Kansas', 'Utah', 'Florida', 'Delaware', 'West Virginia'])].sample(20)
print("# unique values for spend 2021 for Utah, Kansas, Delaware, and West Virginia:", 
      combined_dataset[combined_dataset['State Name'].isin(['Utah', 'Kansas', 'Delaware', 'West Virginia'])]['spend_2021'].unique())
print("# unique values for Annual Checkup for Florida:", 
      combined_dataset[combined_dataset['State Name'].isin(['Florida'])]['Annual Checkup'].unique())

# Note: Kansas, Utah, Delaware, and West Virginia does not have Spend 2021 variable populated. 
## Note2: Florida doesn't have majority of health outcomes populated.


# unique values for spend 2021 for Utah, Kansas, Delaware, and West Virginia: [nan]
# unique values for Annual Checkup for Florida: [nan]


##### Dataset with removal of any missing value after removing Spend 2021 feature

In [22]:
# It seems resonable to drop 'spend_2021' to get Kansas, Utah, Delaware, and West Virginia back in the data.
# However, I can't see a path to include Florida.
# We can probably take it as a separate case and verify our findings from rest of the data.
# Giving -99 values to missing urban_percentage as we don't want to drop ZIPs just because of this column
combined_dataset_nona = combined_dataset.drop(columns=["spend_2021"]).dropna(
    subset=combined_dataset.columns.difference(
        ["Percent_Urban", "Urban+Rural", "Urban", "spend_2021"]
    )
)
combined_dataset_nona.shape

(25315, 82)

In [23]:
# % Missing values for each column. Function defined in helper functions notebook
#perc_missing_num(combined_dataset_nona)

In [24]:
states_in_nona=combined_dataset_nona['State Name'].unique()
print("States not present in nona but present in original dataset")
for state in states_in_original:
    if state not in states_in_nona: print(state)

States not present in nona but present in original dataset
Puerto Rico
nan
Florida
American Samoa
Northern Mariana Islands
Trust Territories
Guam
Virgin Islands


In [25]:
#Summary of #ZIPs by State to see national coverage. Unfortunately Florida is not included
#combined_dataset_nona[['State Name', 'ZIP']].groupby(by='State Name').count()

In [26]:
# Creating a separate dataframe for Florida with only those features that are available for Florida
# Giving -99 values to missing urban_percentage as we don't want to drop ZIPs just because of this column
fl_allna_dropped = combined_dataset[
    combined_dataset["State Name"].isin(["Florida"])
].dropna(axis=1, how="all")
combined_dataset_nona_florida = fl_allna_dropped.dropna(
    subset=fl_allna_dropped.columns.difference(
        ["Percent_Urban", "Urban+Rural", "Urban", "spend_2021"]
    )
)
combined_dataset_nona_florida.shape

(892, 54)

Insights:
<ol>
<li>29 health outcomes features don't have data for 8,068 or 20% ZIPs. Most of these ZIPs should be 0 or low population as per Census procedure documented in the notes for this dataset.</li>
<li>Another 21 features don't have data for at least 7,000 or more ZIPs.</li>
<li>Another 18 features don't have data for at least 5,704 or more ZIPs.</li>
<li>Removing all ZIPs with missing data leaves us with 24,089 ZIPs (61%) out of 39,503 ZIPs in the original combined dataset. But it will exclude Kansas, Utah and Florida all together.</li>
<li>Kansas, Utah, Delaware and West Virginia do not have Spend 2021 variable populated. Florida doesn't have majority of health outcomes populated.</li>
<li>Recommend to  drop spend 2021 to get Kansas, Utah, Delaware and West Virginia back in the data. </li>
<li>However, I can't see a path to include Florida. We can probably take it as a separate case and verify our findings from rest of the data.</li>
<li>This leaves us with 25,321 ZIPs with 80 features in the primary dataset and 893 ZIPs with 51 features in the Florida dataset. </li>
<li>This seems like a reasonable sample size with national coverage. Dropping ZIPs with missing data will remove the noise.</li>
<li>Recommend to move ahead with 25K ZIPs for further analysis.</li>
</ol>

##### Saving dataset without any missing information for any ZIP

In [27]:
#Saving dataset without any missing information
combined_dataset_nona.to_pickle(output_path+"\\combined_dataset_nona.pkl")
combined_dataset_nona.to_csv(output_path+"\\combined_dataset_nona.csv")

#Saving separate dataset for Florida
combined_dataset_nona_florida.to_pickle(output_path+"\\combined_dataset_nona_florida.pkl")
combined_dataset_nona_florida.to_csv(output_path+"\\combined_dataset_nona_florida.csv")

In [29]:
#25321 ZIPs / 82 cols in rest of US, 893 ZIPs / 54 cols in florida
combined_dataset_nona.shape, combined_dataset_nona_florida.shape

((25315, 82), (892, 54))